In [1]:
import math
from icalendar import Calendar, Event, vCalAddress, vText
from pathlib import Path
import os
import pytz

In [2]:
from datetime import datetime

In [3]:
from mvg_api import *
import json
import pprint

def generate_route(start, dest, time=None, arrival_time=False):
    
    # transfer the start and dest address to latitude, longitude coordinate
    start_coordinates = (get_locations(start)[0]['latitude'], get_locations(start)[0]['longitude'])
    dest_coordinates = (get_locations(dest)[0]['latitude'], get_locations(dest)[0]['longitude'])
    
    # get the routes
    routes = get_route(start_coordinates, dest_coordinates, time=time, arrival_time=arrival_time, 
    max_walk_time_to_start=None, max_walk_time_to_dest=None, change_limit=None, ubahn=True, bus=True, tram=True, sbahn=True)

    ## output variables
    route_choose = None

    # time = departure time
    if arrival_time == False:
        route_choose = 0

    # time = arrival time
    else:
        for j in range(len(routes)):
            end_time = routes[-1-j]["arrival_datetime"]
            if end_time < time:
                route_choose = -1-j
                break

    # 	start_time:
    start_time = routes[route_choose]["departure_datetime"]
    # 	end_time: 
    end_time = routes[route_choose]["arrival_datetime"]

    #   model: U-Bahn | S-Bahn | Bus | Tram | Walking "connectionPartType" "FOOTWAY" or "TRANSPORTATION"
    model = []
    station_line = []
    platform = []
    departures = []
    arrivals = []
    name_of_station_start = None
    lock1 = 0
    lock2 = 0
    name_of_station_dest = None
    
    for i in range(len(routes[route_choose]["connectionPartList"])):
        model.append(routes[route_choose]["connectionPartList"][i]["connectionPartType"])

        if "label" in routes[route_choose]["connectionPartList"][i]:
            station_line.append(routes[route_choose]["connectionPartList"][i]["label"])

        if "departurePlatform" in routes[route_choose]["connectionPartList"][i]:
            platform.append(routes[route_choose]["connectionPartList"][i]["departurePlatform"])
        
        if "departure" in routes[route_choose]["connectionPartList"][i]:
            departures.append(datetime.fromtimestamp(routes[route_choose]["connectionPartList"][i]["departure"] / 1000))
        
        if "arrival" in routes[route_choose]["connectionPartList"][i]:
            arrivals.append(datetime.fromtimestamp(routes[route_choose]["connectionPartList"][i]["arrival"] / 1000))

        if "name" in routes[route_choose]["connectionPartList"][i]["from"]:
            if lock1 == 0:
                name_of_station_start = routes[route_choose]["connectionPartList"][i]["from"]["name"]
                lock1 = 1

        if "name" in routes[route_choose]["connectionPartList"][-1-i]["to"]:
            if lock2 == 0:
                name_of_station_dest = routes[route_choose]["connectionPartList"][-1-i]["to"]["name"]
                lock2 = 1
	    
#     route_json = pprint.pformat(routes[-1]).replace("'", '"')
#     with open('route.json', 'w') as f:
#         f.write(route_json)

    return {"start_time": start_time,
            "end_time": end_time,
            "model": model,
            "station_line": station_line,
            "platform": platform,
            "start_station": name_of_station_start,
            "end_station": name_of_station_dest,
            "departures": departures,
            "arrivals": arrivals,
            "duration": end_time - start_time
    }



In [4]:
class TravelPlan:
    def __init__(self, model, station_line, platform, start_station, end_station, departures, arrivals):
        self.model = model
        self.station_line = station_line
        self.platform = platform
        self.start_station = start_station
        self.end_station = end_station
        self.departures = departures
        self.arrivals = arrivals
    
    def print_plan(self):
        print("Start station:", self.start_station)
        for i in range(len(self.station_line)):
            print(self.station_line[i], "-", self.platform[i], "(", self.departures[i], "-", self.arrivals[i], ")")
        print("End station:", self.end_station)
    
    def convert_to_json(self):
        return {"model": self.model, "station_line": self.station_line, "platform": self.platform,
               "start_station": self.start_station, "end_station": self.end_station,
               "departures": self.departures, "arrivals": self.arrivals}

In [9]:
class Activity:
    def __init__(self, name=None, start_time=None, end_time=None, duration=None, location=None, priority=None, travel_plan=None):
        self.name = name
        if (start_time == None) and (end_time != None):
            self.start_time = end_time - duration
        else:
            self.start_time = start_time
        if (end_time == None) and (start_time != None):
            self.end_time = start_time + duration
        else:
            self.end_time = end_time
        if (duration == None) and (start_time != None) and (end_time != None):
            self.duration = end_time - start_time
        else:
            self.duration = duration
        self.location = location
        self.priority = priority
        self.travel_plan = travel_plan

class EmptySlot:
    def __init__(self, start_time, end_time, prev_activity, next_activity):
        self.start_time = start_time
        self.end_time = end_time
        self.prev_activity = prev_activity
        self.next_activity = next_activity

class Schedule:
    def __init__(self):
        self.activities = []
    
    def add_activity(self, activity):
        self.activities.append(activity)
        self._sort_based_on_start_time()

    def _sort_based_on_start_time(self):
        self.activities.sort(key=lambda x: x.start_time)
    
    def find_empty_slots(self):  
        empty_slots = []
        for i in range(len(self.activities)-1):
            prev_act = self.activities[i]
            next_act = self.activities[i+1]
            duration = (next_act.start_time - prev_act.end_time).seconds
            if duration > 0:
                empty_slots.append(EmptySlot(start_time=prev_act.end_time, end_time=next_act.start_time, prev_activity=prev_act, next_activity=next_act))
        return empty_slots
    
    def print_schedule(self):
        for act in self.activities:
            print(act.start_time, "-", act.end_time, ":", act.name)
            if act.travel_plan != None:
                print("--- Travel plan ---")
                act.travel_plan.print_plan()
    
    def convert_to_json(self):
        output = []
        for act in self.activities:
            act_json = {"name": act.name, "start_time": act.start_time, "end_time": act.end_time,
                           "duration": act.duration, "location": act.location}
            if act.travel_plan != None:
                act_json["travel_plan"] = act.travel_plan.convert_to_json()
            output.append(act_json)
        return output

def make_activity_objects(activities):
    processed_activities = []
    for act in activities:
        name = None
        if act["name"] != None:
            name = act["name"]
        location = None
        if act["location"] != None:
            location = act["location"]
        start_time = None
        if act["start_time"] != None:
            start_time = datetime.strptime(act["start_time"], "%Y-%m-%d %H:%M:%S")
        end_time = None
        if act["end_time"] != None:
            end_time = datetime.strptime(act["end_time"], "%Y-%m-%d %H:%M:%S")
        duration = None
        if act["duration"] != None:
            duration = timedelta(minutes=act["duration"])
        processed_act = Activity(name=name, start_time=start_time, end_time=end_time, duration=duration, location=location)
        processed_activities.append(processed_act)
    return processed_activities

def extract_uni_activities(calendar_file):
    file = open(calendar_file, 'rb')
    ecal = Calendar.from_ical(file.read())
    uni_activities = []
    for component in ecal.walk():
        if component.name == 'VEVENT':
            if component.decoded("dtstart").strftime("%Y-%m-%d") == datetime.strptime('2022-11-30', "%Y-%m-%d").strftime("%Y-%m-%d"):
                name = str(component.get('summary'))
                start_time = component.decoded("dtstart").replace(tzinfo=None)
                end_time = component.decoded("dtend").replace(tzinfo=None)
                uni_activities.append(Activity(name=name, start_time=start_time, end_time=end_time, location="Garching Forschungszentrum"))
    return uni_activities

def assign_activity_priorities(activities):
    for activity in activities:
        if (((activity.start_time != None) and (activity.end_time != None)) or
        ((activity.duration != None) and (activity.end_time != None)) or
        ((activity.duration != None) and (activity.start_time != None)) or
        ((activity.duration != None) and (activity.start_time != None) and (activity.end_time != None))):
            activity.priority = 1
        elif (activity.start_time == None) and (activity.end_time == None) and (activity.duration != None) and (activity.location != None):
            activity.priority = 2
        elif (activity.start_time == None) and (activity.end_time == None) and (activity.duration != None) and (activity.location == None):
            activity.priority = 3
        else:
            activity.priority = 4
    activities.sort(key=lambda x: x.priority)

def get_route_duration(loc1, loc2):
    return 0

def create_schedule(remaining_activities, uni_activities=None, travel_activities=None):
    schedule = Schedule()
    
    # Insert uni activities to schedule
    if uni_activities != None:
        for act in uni_activities:
            schedule.add_activity(act)
        
    # Insert travel activities to schedule
    if travel_activities != None:
        for act in travel_activities:
            schedule.add_activity(act)
        
    # Assign priority to remaining activities
    assign_activity_priorities(remaining_activities)
    
    # Schedule remaining activities based on priority
    for act in remaining_activities:
        empty_slots = schedule.find_empty_slots()
        
        if act.priority == 1:
            schedule.add_activity(act)
            
        elif act.priority == 2:
            min_total_duration = math.inf
            min_activity = None
            for slot in empty_slots:
                route = None
                
                travel_time1 = 0
                if slot.prev_activity.location != act.location:
                    route = generate_route(start=slot.prev_activity.location, dest=act.location, time=slot.start_time, arrival_time=False)
                    travel_time1 = route["duration"]
                    route = TravelPlan(model=route["model"], station_line=route["station_line"], platform=route["platform"], start_station=route["start_station"], end_station=route["end_station"], departures=route["departures"], arrivals=route["arrivals"])
                    
                travel_time2 = 0
                if act.location != slot.next_activity.location:
                    route = generate_route(start=act.location, dest=slot.next_activity.location, time=slot.start_time + act.duration, arrival_time=False)
                    travel_time2 = route["duration"]
                    route = TravelPlan(model=route["model"], station_line=route["station_line"], platform=route["platform"], start_station=route["start_station"], end_station=route["end_station"], departures=route["departures"], arrivals=route["arrivals"])
                               
                total_duration = (travel_time1 + act.duration + travel_time2).seconds
                if total_duration < min_total_duration:
                    min_total_duration = total_duration
                    min_activity = Activity(name=act.name, start_time=slot.prev_activity.end_time, duration=act.duration, location=act.location, travel_plan=route)
            
            schedule.add_activity(min_activity)
        
        elif act.priority == 3:
            for slot in empty_slots:
                slot_duration = (slot.end_time - slot.start_time).seconds
                if act.duration.seconds < slot_duration:
                    schedule.add_activity(Activity(name=act.name, start_time=slot.start_time, duration=act.duration))
                    break
    
    return schedule

In [10]:
def get_schedule(request):
     # Get to-do list
    todo_activities = make_activity_objects(request["data"])

    # Get uni activities
    uni_activities = extract_uni_activities('personal_20221119_172719.ics')

    # Get travel activities
    departure_time = request["departure_time"]
    departure_place = request["departure_place"]
    arrival_time = request["arrival_time"]
    arrival_place = request["arrival_place"]
    departure_route = generate_route(start=departure_place, dest=uni_activities[0].location, time=departure_time, arrival_time=False)
    departure_travel_plan = TravelPlan(model=departure_route["model"], station_line=departure_route["station_line"], platform=departure_route["platform"], start_station=departure_route["start_station"], end_station=departure_route["end_station"], departures=departure_route["departures"], arrivals=departure_route["arrivals"])
    departure_activity = Activity(name="Departure", start_time=departure_route["start_time"], end_time=departure_route["end_time"], location=departure_route["end_station"], travel_plan=departure_travel_plan)
    arrival_route = generate_route(start=uni_activities[-1].location, dest=arrival_place, time=arrival_time, arrival_time=True)
    arrival_travel_plan = TravelPlan(model=arrival_route["model"], station_line=arrival_route["station_line"], platform=arrival_route["platform"], start_station=arrival_route["start_station"], end_station=arrival_route["end_station"], departures=arrival_route["departures"], arrivals=arrival_route["arrivals"])
    arrival_activity = Activity(name="Arrival", start_time=arrival_route["start_time"], end_time=arrival_route["end_time"], location=arrival_route["start_station"], travel_plan=arrival_travel_plan)
    travel_activities = [departure_activity, arrival_activity]

    # Create schedule
    schedule = create_schedule(todo_activities, uni_activities, travel_activities)
    schedule.print_schedule()

    return schedule.convert_to_json()

In [11]:
from datetime import datetime, timedelta

In [12]:
request = {"data": [{"name": "Reading", "location": None, "start_time": None, "end_time": None, "duration": 60},
                   {"name": "Exercise", "location": "Laim", "start_time": None, "end_time": None, "duration": 30}],
          "departure_time": datetime.strptime("2022-11-30 06:00:00", "%Y-%m-%d %H:%M:%S"),
          "arrival_time": datetime.strptime("2022-11-30 23:00:00", "%Y-%m-%d %H:%M:%S"),
          "departure_place": "Hohenzollernplatz",
          "arrival_place": "Garching"}
schedule_json = get_schedule(request)

2022-11-30 06:03:00 - 2022-11-30 06:41:00 : Departure
--- Travel plan ---
Start station: Hohenzollernplatz
53 -  ( 2022-11-30 06:03:00 - 2022-11-30 06:08:00 )
U6 - Gleis 1 ( 2022-11-30 06:08:00 - 2022-11-30 06:13:00 )
End station: Garching-Forschungszentrum
2022-11-30 06:41:00 - 2022-11-30 07:11:00 : Exercise
--- Travel plan ---
Start station: Laim
S1 - 2 ( 2022-11-30 07:09:00 - 2022-11-30 07:10:00 )
U6 - U-Bahn Gleis 1 ( 2022-11-30 07:10:00 - 2022-11-30 07:20:00 )
End station: Garching-Forschungszentrum
2022-11-30 08:00:00 - 2022-11-30 09:00:00 : Übung zu Quantentechnologie UE, Gruppe 1 Johannes Früh
2022-11-30 09:00:00 - 2022-11-30 11:00:00 : Quantentechnologie VO, Standardgruppe
2022-11-30 11:00:00 - 2022-11-30 12:00:00 : Reading
2022-11-30 13:00:00 - 2022-11-30 15:00:00 : Image Guided Surgery (IN2286) VI, Standardgruppe
2022-11-30 17:00:00 - 2022-11-30 18:30:00 : Deutsch als Fremdsprache A2.1 SE, Keza GERMAN MATTERS (GAR Präsenz)
2022-11-30 22:40:00 - 2022-11-30 22:51:00 : Arrival


In [10]:
schedule_json

[{'name': 'Departure',
  'start_time': datetime.datetime(2022, 11, 30, 6, 3),
  'end_time': datetime.datetime(2022, 11, 30, 6, 41),
  'duration': datetime.timedelta(seconds=2280),
  'location': 'Garching-Forschungszentrum',
  'travel_plan': {'model': ['FOOTWAY',
    'TRANSPORTATION',
    'TRANSPORTATION',
    'FOOTWAY'],
   'station_line': ['53', 'U6'],
   'platform': ['', 'Gleis 1'],
   'start_station': 'Hohenzollernplatz',
   'end_station': 'Garching-Forschungszentrum'}},
 {'name': 'Reading',
  'start_time': datetime.datetime(2022, 11, 30, 6, 41),
  'end_time': datetime.datetime(2022, 11, 30, 7, 41),
  'duration': datetime.timedelta(seconds=3600),
  'location': None},
 {'name': 'Übung zu Quantentechnologie UE, Gruppe 1 Johannes Früh',
  'start_time': datetime.datetime(2022, 11, 30, 8, 0),
  'end_time': datetime.datetime(2022, 11, 30, 9, 0),
  'duration': datetime.timedelta(seconds=3600),
  'location': 'Garching Forschungszentrum'},
 {'name': 'Quantentechnologie VO, Standardgruppe',
 

In [9]:
start = "Garching Forschungszentrum"
dest = "Garching"
get_route(start=(get_locations(start)[0]['latitude'], get_locations(start)[0]['longitude']), dest=(get_locations(dest)[0]['latitude'], get_locations(dest)[0]['longitude']))

[{'zoomNoticeFrom': False,
  'zoomNoticeTo': False,
  'zoomNoticeFromEscalator': False,
  'zoomNoticeToEscalator': False,
  'zoomNoticeFromElevator': False,
  'zoomNoticeToElevator': False,
  'from': {'type': 'address',
   'latitude': 48.26486,
   'longitude': 11.671218,
   'poi': False},
  'to': {'type': 'address',
   'latitude': 48.249388,
   'longitude': 11.65256,
   'poi': False},
  'departure': 1668905040000,
  'arrival': 1668905460000,
  'connectionPartList': [{'from': {'type': 'address',
     'latitude': 48.26486,
     'longitude': 11.671218,
     'poi': False},
    'to': {'type': 'station',
     'latitude': 48.26486,
     'longitude': 11.67123,
     'id': 'de:09184:460',
     'divaId': 460,
     'place': 'Garching (b München)',
     'name': 'Garching-Forschungszentrum',
     'hasLiveData': True,
     'hasZoomData': True,
     'products': ['BUS', 'UBAHN'],
     'efaLon': 11.671361,
     'efaLat': 48.264808,
     'link': 'GF',
     'tariffZones': '2',
     'occupancy': 'UNKNOWN',